In [1]:
import tensorflow as tf
from keras.models import Sequential
import keras.layers as layers
from spp.SpatialPyramidPooling import SpatialPyramidPooling

import random

import numpy as np
from typing import Tuple
from dataset_functions import obtain_image_list

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [2]:
# Locations
FOLDER_LOCATION = "E:\Downloads\Cats-vs-Dogs-all"
TEST_FOLDER = "test"
TRAIN_FOLDER = "train"

# Aims
## Keras
I aim to generate a CNN that is capable of identifying the difference between cats and dogs


Layer 1:
CNN
Number of channels: 200
Kernel: [3, 3]
activation: RELU
[2, 2]

Layer 2:
CNN
Number of channels: 200
Kernel: [3, 3]
activation: RELU
[2, 2]

Layer 4:
CNN
Number of channels: 100
Kernel: [3, 3]
activation: RELU
[2, 2]

Potential Global Pooling layer

Layer 5:
FC
nodes: 300
dropout 0.4
activation: RELU

Layer 6:
FC
nodes: 200
dropout: 0.4
activation: RELU

Layer 7:
FC
nodes: 125
dropout: 0.4
activation: RELU

Layer 8:
FC
nodes: 2
activation: Linear

Use Logits to obtain system accuracy


In [3]:
# Load in the training data and shuffle it with KFolds and a validation dataset

train_dogs_labels, _, train_dogs_images = obtain_image_list("dog", train=True)
train_cats_labels, _, train_cats_images = obtain_image_list("cat", train=True)

train_all_labels = train_dogs_labels + train_cats_labels
train_all_images = train_dogs_images + train_cats_images

100%|██████████| 12500/12500 [00:49<00:00, 253.92it/s]


In [4]:
# Convert the lists to a numpy array and shuffle the array
train_all_labels_np = np.reshape(np.array(train_all_labels), (-1, 1))
train_all_images_np = np.array(train_all_images)

one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoderin_all_labels_encoded = one_hot_encoder.fit_transform(train_all_labels_np)
print(one_hot_encoderin_all_labels_encoded)

# Shuffle the arrays
shuffle_args = list(range(len(train_all_labels)))
random.shuffle(shuffle_args)

train_all_labels_shuffled_np = one_hot_encoderin_all_labels_encoded[shuffle_args, :]
train_all_images_shuffled_np = train_all_images_np[shuffle_args]
print(train_all_labels_shuffled_np)

X_train, X_validate, y_train, y_validate = train_test_split(train_all_images_shuffled_np, 
                                                            train_all_labels_shuffled_np, 
                                                            test_size=0.10)

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [5]:
X_train.shape

(22500,)

In [6]:
LAYER_1_CNN = 200
LAYER_2_CNN = 200
LAYER_3_CNN = 100

LAYER_1_FC = 60
LAYER_2_FC = 50
LAYER_3_FC = 50
LAYER_OUTPUT = 2

model = Sequential()
model.add(layers.Convolution2D(filters=8, kernel_size=(3,3), strides=(1,1), padding='same', activation="relu", input_shape=(None, None, 3)))
model.add(layers.Convolution2D(filters=12, kernel_size=(5,5), strides=(1,1), padding='same', activation="relu"))
model.add(layers.Convolution2D(filters=6, kernel_size=(3,3), strides=(1,1), padding='same', activation="relu"))
model.add(SpatialPyramidPooling([3]))
model.add(layers.Dense(LAYER_1_FC, activation="relu", name="FC_1"))
model.add(layers.Dropout(0.50))
model.add(layers.Dense(LAYER_2_FC, activation="relu", name="FC_2"))
model.add(layers.Dropout(0.50))
model.add(layers.Dense(LAYER_OUTPUT, activation="softmax", name="FC_output"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, None, None, 8)     224       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 12)    2412      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 6)     654       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, None, None, 6)     330       
_________________________________________________________________
spatial_pyramid_pooling_1 (S (None, 60)                0         
_________________________________________________________________
FC_1 (Dense)                 (None, 50)                3050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
__________

In [15]:
def generator(input_data, input_label):
    while True:
        print("\n")
        for array_val, label in zip(input_data, input_label):
            yield np.expand_dims(array_val/255, axis=0), np.reshape(label, (1, -1))

In [16]:
model.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])
# Trains for 5 epochs.

In [18]:
model.fit_generator(generator(X_train, y_train), steps_per_epoch=X_train.shape[0], epochs=10)

Epoch 1/10




  588/22500 [..............................] - ETA: 22:47 - loss: 0.7065 - acc: 0.0000e+ - ETA: 12:03 - loss: 0.6776 - acc: 0.5000   - ETA: 10:31 - loss: 0.6849 - acc: 0.42 - ETA: 9:56 - loss: 0.6767 - acc: 0.6000 - ETA: 9:18 - loss: 0.6760 - acc: 0.615 - ETA: 9:05 - loss: 0.6742 - acc: 0.625 - ETA: 9:01 - loss: 0.6805 - acc: 0.578 - ETA: 8:48 - loss: 0.6824 - acc: 0.590 - ETA: 8:40 - loss: 0.6758 - acc: 0.640 - ETA: 8:40 - loss: 0.6744 - acc: 0.642 - ETA: 8:43 - loss: 0.6766 - acc: 0.633 - ETA: 8:44 - loss: 0.6817 - acc: 0.606 - ETA: 8:47 - loss: 0.6775 - acc: 0.628 - ETA: 8:49 - loss: 0.6856 - acc: 0.578 - ETA: 8:52 - loss: 0.6866 - acc: 0.575 - ETA: 8:47 - loss: 0.6912 - acc: 0.558 - ETA: 8:45 - loss: 0.6903 - acc: 0.543 - ETA: 8:45 - loss: 0.6962 - acc: 0.510 - ETA: 8:42 - loss: 0.6982 - acc: 0.500 - ETA: 8:37 - loss: 0.7009 - acc: 0.490 - ETA: 8:35 - loss: 0.6975 - acc: 0.517 - ETA: 8:32 - loss: 0.6976 - acc: 0.524 - ETA: 8:33 - loss: 0.6970 - acc: 0.531 - ETA: 8:33 - loss: 0.6977

 1188/22500 [>.............................] - ETA: 8:02 - loss: 0.6958 - acc: 0.505 - ETA: 8:02 - loss: 0.6958 - acc: 0.506 - ETA: 8:02 - loss: 0.6958 - acc: 0.505 - ETA: 8:02 - loss: 0.6957 - acc: 0.508 - ETA: 8:02 - loss: 0.6957 - acc: 0.509 - ETA: 8:01 - loss: 0.6958 - acc: 0.506 - ETA: 8:01 - loss: 0.6957 - acc: 0.509 - ETA: 8:01 - loss: 0.6957 - acc: 0.506 - ETA: 8:00 - loss: 0.6957 - acc: 0.507 - ETA: 8:00 - loss: 0.6957 - acc: 0.508 - ETA: 8:00 - loss: 0.6957 - acc: 0.505 - ETA: 8:00 - loss: 0.6958 - acc: 0.504 - ETA: 8:00 - loss: 0.6957 - acc: 0.507 - ETA: 7:59 - loss: 0.6957 - acc: 0.506 - ETA: 7:59 - loss: 0.6958 - acc: 0.505 - ETA: 7:59 - loss: 0.6958 - acc: 0.506 - ETA: 7:59 - loss: 0.6958 - acc: 0.507 - ETA: 7:59 - loss: 0.6957 - acc: 0.506 - ETA: 7:59 - loss: 0.6957 - acc: 0.505 - ETA: 7:59 - loss: 0.6956 - acc: 0.506 - ETA: 7:58 - loss: 0.6955 - acc: 0.508 - ETA: 7:58 - loss: 0.6956 - acc: 0.507 - ETA: 7:58 - loss: 0.6956 - acc: 0.506 - ETA: 7:58 - loss: 0.6955 - acc: 0

 1783/22500 [=>............................] - ETA: 7:46 - loss: 0.6958 - acc: 0.503 - ETA: 7:46 - loss: 0.6958 - acc: 0.505 - ETA: 7:46 - loss: 0.6957 - acc: 0.505 - ETA: 7:46 - loss: 0.6958 - acc: 0.504 - ETA: 7:46 - loss: 0.6958 - acc: 0.503 - ETA: 7:46 - loss: 0.6958 - acc: 0.504 - ETA: 7:46 - loss: 0.6958 - acc: 0.503 - ETA: 7:46 - loss: 0.6958 - acc: 0.504 - ETA: 7:46 - loss: 0.6958 - acc: 0.505 - ETA: 7:46 - loss: 0.6957 - acc: 0.505 - ETA: 7:46 - loss: 0.6957 - acc: 0.505 - ETA: 7:46 - loss: 0.6957 - acc: 0.504 - ETA: 7:46 - loss: 0.6956 - acc: 0.505 - ETA: 7:46 - loss: 0.6956 - acc: 0.506 - ETA: 7:46 - loss: 0.6957 - acc: 0.506 - ETA: 7:46 - loss: 0.6956 - acc: 0.505 - ETA: 7:46 - loss: 0.6956 - acc: 0.506 - ETA: 7:46 - loss: 0.6956 - acc: 0.505 - ETA: 7:46 - loss: 0.6956 - acc: 0.505 - ETA: 7:45 - loss: 0.6956 - acc: 0.504 - ETA: 7:45 - loss: 0.6956 - acc: 0.505 - ETA: 7:45 - loss: 0.6956 - acc: 0.505 - ETA: 7:45 - loss: 0.6955 - acc: 0.505 - ETA: 7:45 - loss: 0.6955 - acc: 0

 2369/22500 [==>...........................] - ETA: 7:35 - loss: 0.6953 - acc: 0.510 - ETA: 7:35 - loss: 0.6954 - acc: 0.510 - ETA: 7:35 - loss: 0.6953 - acc: 0.511 - ETA: 7:35 - loss: 0.6953 - acc: 0.510 - ETA: 7:35 - loss: 0.6953 - acc: 0.510 - ETA: 7:35 - loss: 0.6953 - acc: 0.510 - ETA: 7:35 - loss: 0.6952 - acc: 0.510 - ETA: 7:35 - loss: 0.6954 - acc: 0.510 - ETA: 7:35 - loss: 0.6954 - acc: 0.509 - ETA: 7:35 - loss: 0.6953 - acc: 0.509 - ETA: 7:35 - loss: 0.6954 - acc: 0.509 - ETA: 7:35 - loss: 0.6954 - acc: 0.509 - ETA: 7:35 - loss: 0.6953 - acc: 0.509 - ETA: 7:35 - loss: 0.6953 - acc: 0.510 - ETA: 7:35 - loss: 0.6952 - acc: 0.509 - ETA: 7:35 - loss: 0.6953 - acc: 0.509 - ETA: 7:35 - loss: 0.6952 - acc: 0.509 - ETA: 7:35 - loss: 0.6952 - acc: 0.510 - ETA: 7:34 - loss: 0.6952 - acc: 0.510 - ETA: 7:34 - loss: 0.6952 - acc: 0.509 - ETA: 7:34 - loss: 0.6952 - acc: 0.510 - ETA: 7:34 - loss: 0.6952 - acc: 0.510 - ETA: 7:34 - loss: 0.6952 - acc: 0.510 - ETA: 7:34 - loss: 0.6952 - acc: 0

...

20274/22500 [==========================>...] - ETA: 1:02 - loss: 0.5566 - acc: 0.721 - ETA: 1:02 - loss: 0.5565 - acc: 0.721 - ETA: 1:02 - loss: 0.5564 - acc: 0.721 - ETA: 1:02 - loss: 0.5565 - acc: 0.721 - ETA: 1:02 - loss: 0.5564 - acc: 0.721 - ETA: 1:02 - loss: 0.5564 - acc: 0.721 - ETA: 1:02 - loss: 0.5564 - acc: 0.721 - ETA: 1:02 - loss: 0.5565 - acc: 0.721 - ETA: 1:01 - loss: 0.5565 - acc: 0.721 - ETA: 1:01 - loss: 0.5565 - acc: 0.721 - ETA: 1:01 - loss: 0.5565 - acc: 0.721 - ETA: 1:01 - loss: 0.5565 - acc: 0.721 - ETA: 1:01 - loss: 0.5565 - acc: 0.721 - ETA: 1:01 - loss: 0.5565 - acc: 0.721 - ETA: 1:01 - loss: 0.5564 - acc: 0.721 - ETA: 1:01 - loss: 0.5564 - acc: 0.721 - ETA: 1:01 - loss: 0.5564 - acc: 0.721 - ETA: 1:01 - loss: 0.5564 - acc: 0.721 - ETA: 1:01 - loss: 0.5563 - acc: 0.721 - ETA: 1:01 - loss: 0.5564 - acc: 0.721 - ETA: 1:01 - loss: 0.5564 - acc: 0.721 - ETA: 1:01 - loss: 0.5564 - acc: 0.721 - ETA: 1:01 - loss: 0.5564 - acc: 0.721 - ETA: 1:01 - loss: 0.5565 - acc: 0

20900/22500 [==========================>...] - ETA: 48s - loss: 0.5562 - acc: 0.72 - ETA: 48s - loss: 0.5562 - acc: 0.72 - ETA: 48s - loss: 0.5562 - acc: 0.72 - ETA: 48s - loss: 0.5562 - acc: 0.72 - ETA: 48s - loss: 0.5562 - acc: 0.72 - ETA: 48s - loss: 0.5562 - acc: 0.72 - ETA: 48s - loss: 0.5562 - acc: 0.72 - ETA: 48s - loss: 0.5562 - acc: 0.72 - ETA: 48s - loss: 0.5561 - acc: 0.72 - ETA: 48s - loss: 0.5562 - acc: 0.72 - ETA: 48s - loss: 0.5561 - acc: 0.72 - ETA: 48s - loss: 0.5561 - acc: 0.72 - ETA: 48s - loss: 0.5561 - acc: 0.72 - ETA: 48s - loss: 0.5561 - acc: 0.72 - ETA: 47s - loss: 0.5562 - acc: 0.72 - ETA: 47s - loss: 0.5562 - acc: 0.72 - ETA: 47s - loss: 0.5563 - acc: 0.72 - ETA: 47s - loss: 0.5562 - acc: 0.72 - ETA: 47s - loss: 0.5562 - acc: 0.72 - ETA: 47s - loss: 0.5562 - acc: 0.72 - ETA: 47s - loss: 0.5562 - acc: 0.72 - ETA: 47s - loss: 0.5563 - acc: 0.72 - ETA: 47s - loss: 0.5562 - acc: 0.72 - ETA: 47s - loss: 0.5563 - acc: 0.72 - ETA: 47s - loss: 0.5562 - acc: 0.72 - ETA

21532/22500 [===========================>..] - ETA: 35s - loss: 0.5564 - acc: 0.72 - ETA: 35s - loss: 0.5563 - acc: 0.72 - ETA: 34s - loss: 0.5563 - acc: 0.72 - ETA: 34s - loss: 0.5563 - acc: 0.72 - ETA: 34s - loss: 0.5562 - acc: 0.72 - ETA: 34s - loss: 0.5563 - acc: 0.72 - ETA: 34s - loss: 0.5563 - acc: 0.72 - ETA: 34s - loss: 0.5562 - acc: 0.72 - ETA: 34s - loss: 0.5562 - acc: 0.72 - ETA: 34s - loss: 0.5563 - acc: 0.72 - ETA: 34s - loss: 0.5563 - acc: 0.72 - ETA: 34s - loss: 0.5562 - acc: 0.72 - ETA: 34s - loss: 0.5562 - acc: 0.72 - ETA: 34s - loss: 0.5563 - acc: 0.72 - ETA: 34s - loss: 0.5563 - acc: 0.72 - ETA: 34s - loss: 0.5563 - acc: 0.72 - ETA: 34s - loss: 0.5563 - acc: 0.72 - ETA: 34s - loss: 0.5562 - acc: 0.72 - ETA: 33s - loss: 0.5562 - acc: 0.72 - ETA: 33s - loss: 0.5562 - acc: 0.72 - ETA: 33s - loss: 0.5562 - acc: 0.72 - ETA: 33s - loss: 0.5565 - acc: 0.72 - ETA: 33s - loss: 0.5564 - acc: 0.72 - ETA: 33s - loss: 0.5564 - acc: 0.72 - ETA: 33s - loss: 0.5564 - acc: 0.72 - ETA

22167/22500 [============================>.] - ETA: 21s - loss: 0.5564 - acc: 0.72 - ETA: 21s - loss: 0.5564 - acc: 0.72 - ETA: 21s - loss: 0.5564 - acc: 0.72 - ETA: 21s - loss: 0.5563 - acc: 0.72 - ETA: 20s - loss: 0.5563 - acc: 0.72 - ETA: 20s - loss: 0.5563 - acc: 0.72 - ETA: 20s - loss: 0.5564 - acc: 0.72 - ETA: 20s - loss: 0.5564 - acc: 0.72 - ETA: 20s - loss: 0.5564 - acc: 0.72 - ETA: 20s - loss: 0.5564 - acc: 0.72 - ETA: 20s - loss: 0.5564 - acc: 0.72 - ETA: 20s - loss: 0.5564 - acc: 0.72 - ETA: 20s - loss: 0.5564 - acc: 0.72 - ETA: 20s - loss: 0.5563 - acc: 0.72 - ETA: 20s - loss: 0.5563 - acc: 0.72 - ETA: 20s - loss: 0.5563 - acc: 0.72 - ETA: 20s - loss: 0.5563 - acc: 0.72 - ETA: 20s - loss: 0.5563 - acc: 0.72 - ETA: 20s - loss: 0.5563 - acc: 0.72 - ETA: 19s - loss: 0.5563 - acc: 0.72 - ETA: 19s - loss: 0.5563 - acc: 0.72 - ETA: 19s - loss: 0.5562 - acc: 0.72 - ETA: 19s - loss: 0.5562 - acc: 0.72 - ETA: 19s - loss: 0.5562 - acc: 0.72 - ETA: 19s - loss: 0.5562 - acc: 0.72 - ETA

22489/22500 [============================>.] - ETA: 7s - loss: 0.5558 - acc: 0.722 - ETA: 7s - loss: 0.5558 - acc: 0.722 - ETA: 7s - loss: 0.5558 - acc: 0.722 - ETA: 7s - loss: 0.5558 - acc: 0.722 - ETA: 6s - loss: 0.5558 - acc: 0.722 - ETA: 6s - loss: 0.5559 - acc: 0.722 - ETA: 6s - loss: 0.5558 - acc: 0.722 - ETA: 6s - loss: 0.5558 - acc: 0.722 - ETA: 6s - loss: 0.5558 - acc: 0.722 - ETA: 6s - loss: 0.5558 - acc: 0.722 - ETA: 6s - loss: 0.5557 - acc: 0.722 - ETA: 6s - loss: 0.5559 - acc: 0.722 - ETA: 6s - loss: 0.5559 - acc: 0.722 - ETA: 6s - loss: 0.5561 - acc: 0.722 - ETA: 6s - loss: 0.5561 - acc: 0.722 - ETA: 6s - loss: 0.5561 - acc: 0.722 - ETA: 6s - loss: 0.5560 - acc: 0.722 - ETA: 6s - loss: 0.5560 - acc: 0.722 - ETA: 6s - loss: 0.5561 - acc: 0.722 - ETA: 5s - loss: 0.5561 - acc: 0.722 - ETA: 5s - loss: 0.5561 - acc: 0.722 - ETA: 5s - loss: 0.5561 - acc: 0.722 - ETA: 5s - loss: 0.5560 - acc: 0.722 - ETA: 5s - loss: 0.5561 - acc: 0.722 - ETA: 5s - loss: 0.5561 - acc: 0.722 - ETA